In [1]:
import torch
import re
import pandas as pd
from transformers import AutoTokenizer, BitsAndBytesConfig
from transformers import AutoModelForCausalLM
from peft import PeftModel
from torch import cuda
from sql_metadata import Parser
from tqdm import tqdm

/home/vipuser/anaconda3/envs/DTS_SQL/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "/home/LLM_para/deepspeek_code"
# model_name = "mistralai/Mistral-7B-Instruct-v0.2"
ibnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    attn_implementation="flash_attention_2", # use with amper architecture
    torch_dtype=torch.bfloat16,
    #quantization_config=bnb_config, # use when low on memory
    device_map = "auto"
)

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 2/2 [00:12<00:00,  6.30s/it]


In [3]:
adapter_path = "./final_checkpoint_SFT_deepspeek_Myidea"
model = PeftModel.from_pretrained(model,adapter_path ,torch_dtype = torch.bfloat16)
model = model.merge_and_unload()
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
tokenizer.encode(' ;')

[32013, 6203]

In [5]:
# 拆解训练集
# 提供每次训练token数
from tqdm import tqdm
import copy 
import re
max_token = 1800
accept_toekn = max_token - 150
#获取REF表数据
tab_str = ""
schema_slicing = []
df = pd.read_csv("./mydataset_new/table_schema_Reference_cropped.csv",encoding="utf-8")
for index, row in tqdm(df.iterrows(), total=len(df)):
    table_group = row['Reference_group']
    tab_str_elder = copy.deepcopy(tab_str)
    tab_str += table_group + "\n"
    check_token = tokenizer(f"{tab_str}", add_special_tokens=False)
    if len(check_token["input_ids"]) > accept_toekn:
        schema_slicing.append(tab_str_elder)
        tab_str = table_group + "\n"

    
print(len(schema_slicing))


#获取noREF表数据
df = pd.read_csv("./mydataset_new/table_schema_noReference_cropped.csv",encoding="utf-8")
for index, row in tqdm(df.iterrows(), total=len(df)):
    table_group = row['noReference_group']
    tab_str_elder = copy.deepcopy(tab_str)
    tab_str += table_group + "\n"
    check_token = tokenizer(f"{tab_str}", add_special_tokens=False)
    if len(check_token["input_ids"]) > accept_toekn:
#         print("+++++++++++")
        schema_slicing.append(tab_str_elder)
        tab_str = table_group + "\n"
        
        
if len(tab_str) > 0:
    schema_slicing.append(tab_str)

print(len(schema_slicing))


create_table_pattern = re.compile(r'CREATE TABLE `[^`]+` \([^;]+\);')
table_name_group = []
test_total = set()
count = 0
for piece in schema_slicing:
    create_table_statements = create_table_pattern.findall(piece)
    temp_list = []
    for table in create_table_statements:
        table_name = re.search(r"CREATE TABLE `([^`]+)`", table).group(1)
        temp_list.append(table_name.lower())
        test_total.add(table_name.lower())
        count+=1
    print(temp_list)
    table_name_group.append(temp_list)
# print(count)
print(len(test_total))
#     table_name_group.append()

100%|████████████████████████████████████████████████████████████████████████████████| 142/142 [00:00<00:00, 483.28it/s]


6


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 518.70it/s]

8
['manufacturers', 'products', 'student', 'plays_games', 'sportsinfo', 'actor', 'entrepreneur', 'people', 'ref_hotel_star_ratings', 'ref_attraction_types', 'hotels', 'tourist_attractions', 'street_markets', 'shops', 'museums', 'royal_family', 'theme_parks', 'visits', 'photos', 'staff', 'tourist_attraction_features', 'wrestler', 'elimination']
['business', 'category', 'checkin', 'neighbourhood', 'review', 'tip', 'ref_detention_type', 'ref_incident_type', 'addresses', 'students', 'teachers', 'assessment_notes', 'behavior_incident', 'detention', 'student_addresses', 'students_in_detention', 'film', 'film_market_estimation', 'catalogs']
['catalog_structure', 'catalog_contents', 'catalog_contents_additional_attributes', 'routes', 'airports', 'stadium', 'game', 'injury_accident', 'physician', 'department', 'affiliated_with', 'trained_in', 'patient', 'nurse', 'appointment', 'prescribes', 'block', 'room', 'on_call', 'stay']
['undergoes', 'buildings', 'office_locations', 'region', 'party', 'me

In [6]:
from transformers import StoppingCriteria
# from typing import List
# from transformers.generation.stopping_criteria import StoppingCriteria, StoppingCriteriaList,STOPPING_CRITERIA_INPUTS_DOCSTRING, add_start_docstrings
class EosListStoppingCriteria(StoppingCriteria):
    def __init__(self, eos_sequence = [6203]):
    # 设置在第一个';'处停止      
        self.eos_sequence = eos_sequence

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        last_ids = input_ids[:,-len(self.eos_sequence):].tolist()
        return self.eos_sequence in last_ids
    
def append_string_to_file(text, file_path):
  with open(file_path, 'a') as file:
      file.write(text + '\n')

def remove_spaces(text):
  return re.sub(r'\s+', ' ', text)

def call_mistral(inputs):
    output_tokens = model.generate(inputs, max_new_tokens=30, do_sample=False, pad_token_id=tokenizer.eos_token_id, eos_token_id=tokenizer.eos_token_id, stopping_criteria = [EosListStoppingCriteria()])
#     output_tokens = model.generate(inputs, max_new_tokens=250, do_sample=False, pad_token_id=tokenizer.eos_token_id, eos_token_id=tokenizer.eos_token_id, stopping_criteria = stopping_criteria)
    return tokenizer.decode(output_tokens[0][len(inputs[0]):], skip_special_tokens=True)

In [7]:
def get_input_value(question_pad, selected_table_pad, schema_pad):
    input_value = f"I want you to act as a relation extraction robot for a sample SQL table. You need to return the tables related to the user's input question and tables.##Question:{question_pad} ###selected_table: {selected_table_pad}.Below are instructions describing the relationship between tables. Please write a response that appropriately completes the request. \n##instruction:{schema_pad}"
    return input_value 

In [8]:
df = pd.read_csv("./mydataset_new/validation_dataset_formatted_cropped.csv",encoding="utf-8")
outer_index = 0
import re
pattern = re.compile(r'-- Tables: ([\w, ]+);')
results = []
#  schema_slicing
import time 
start_time = time.time()
for index, row in tqdm(df.iterrows(), total=len(df)):
    question = row['question']
    ref_tables = row['correct_tables']
    selected_table_set = set()
    result_count = 0
    for schema in schema_slicing:
        result_count += 1
        if len(selected_table_set) > 0:
            selected_table =  ', '.join(list(selected_table_set))
        else:
            selected_table = "None"
#         instruction = get_instruction(schema)
#         input_value = get_input_value(question,selected_table)
        input_value = get_input_value(question, selected_table, schema)
        messages = [{"role": "user", "content": input_value.strip()}]
        inputs = tokenizer.apply_chat_template(messages, return_tensors="pt",add_generation_prompt=True,tokenize = True).to(model.device)
        response = call_mistral(inputs)
#         print(response)
        if "#None#" not in response:
            match = re.search(pattern, response)
            if match:
            # 提取匹配的表名字符串
                table_names_str = match.group(1)
                table_names = table_names_str.split(', ')
                for item in table_names:
                     selected_table_set.add(item)
            else:
                predict_table = "None"
    if len(selected_table_set) > 0:
        total_selected_table =  ', '.join(list(selected_table_set))
    else:
        total_selected_table = "None"
    print(f"++++++++++++++++index:{index}++++++++++++++++++++++++")
    print("正确表名：",ref_tables)
    print("预测表名：",total_selected_table)
    print("############################################")
    results.append([ref_tables,total_selected_table])
end_time = time.time()

  1%|▍                                                                                  | 1/178 [00:05<17:28,  5.92s/it]

++++++++++++++++index:0++++++++++++++++++++++++
正确表名： products, manufacturers
预测表名： products, manufacturers
############################################


  1%|▉                                                                                  | 2/178 [00:10<15:10,  5.17s/it]

++++++++++++++++index:1++++++++++++++++++++++++
正确表名： products
预测表名： products
############################################


  2%|█▍                                                                                 | 3/178 [00:15<14:59,  5.14s/it]

...

++++++++++++++++index:175++++++++++++++++++++++++
正确表名： railway
预测表名： railway
############################################


 99%|████████████████████████████████████████████████████████████████████████████████▌| 177/178 [14:43<00:04,  4.81s/it]

++++++++++++++++index:176++++++++++++++++++++++++
正确表名： book
预测表名： book
############################################


100%|█████████████████████████████████████████████████████████████████████████████████| 178/178 [14:48<00:00,  4.99s/it]

++++++++++++++++index:177++++++++++++++++++++++++
正确表名： book, publication
预测表名： book, publication
############################################


In [9]:
total_data = len(df['question'])
total_time = end_time - start_time
# print(total_data)
signal_infernece_time = total_time/total_data
print(signal_infernece_time)

4.993609802106793


In [10]:
# new_df = pd.DataFrame(results, columns = ['predicted_tables','correct_tables','reference_tables','query','question','db_id'])
# new_df.to_csv('new_Prompt.csv', index=False)
#     print(f"++++++++++++++++index:{index}++++++++++++++++++++++++")
#     print("正确表名：",ref_tables)
#     print("预测表名：",total_selected_table)
#     print("############################################")
#     results.append([ref_tables,total_selected_table])
new_df = pd.DataFrame(results, columns = ['correct_tables','predicted_tables'])

In [11]:
total_samples = len(new_df)
total_accuracy = 0
filtered_accuracy = 0
total_precision = 0
total_recall = 0

for index, row in new_df.iterrows():
    
    if not row['predicted_tables'] or pd.isna(row['predicted_tables']):
        continue
    predicted_tables = row['predicted_tables'].split(", ")
    reference_tables = row['correct_tables'].split(", ")
    
    # Convert to lowercase and strip whitespace for comparison
    predicted_tables = [x.lower().replace("--","").replace("**","").strip() for x in predicted_tables]
    reference_tables = [x.lower().strip() for x in reference_tables]
    
    # Calculate accuracy
    if set(predicted_tables) == set(reference_tables):
        total_accuracy += 1
    
    # Calculate precision and recall
    true_positives = len(set(predicted_tables) & set(reference_tables))
    false_positives = len(set(predicted_tables) - set(reference_tables))
    false_negatives = len(set(reference_tables) - set(predicted_tables))

    if true_positives == len(reference_tables):
        filtered_accuracy += 1
    
    if len(predicted_tables) > 0:
        precision = true_positives / (true_positives + false_positives)
        recall = true_positives / (true_positives + false_negatives)
    
    total_precision += precision
    total_recall += recall

# Calculate average precision and recall
avg_precision = total_precision / total_samples
avg_recall = total_recall / total_samples

# Calculate total accuracy
accuracy = total_accuracy / total_samples
filtered_accuracy = filtered_accuracy / total_samples

print("Total Accuracy:", accuracy)
print("Filtered Accuracy:", filtered_accuracy)
print("Average Precision:", avg_precision)
print("Average Recall:", avg_recall)

# new_df.to_csv("generated_schema_links.csv", index=False)

Total Accuracy: 0.7359550561797753
Filtered Accuracy: 0.8539325842696629
Average Precision: 0.8890449438202249
Average Recall: 0.9096441947565544


In [13]:
new_df.to_csv("generated_schema_links.csv", index=False)